In [ ]:
import pandas as pd

In [ ]:
pokemon_dt = pd.read_csv(r"DataBases\\df_pokemon.csv")
pokemon_dt=pokemon_dt.drop(columns=["Species",'Variant','Generation','Rarity','Evolves_from','Has_gender_diff','VGC2022_rules','Monthly Usage (k)','Usage Percent (%)','Monthly Rank'])

pokemon_dt.head(50)

In [ ]:
import pandas as pd
import numpy as np

type_chart=pd.read_csv(r"DataBases\\bridge_type_type_MOVE_EFFECTIVENESS_ON_POKEMON.csv")
type_chart["Damage Multiplier"].fillna(1,inplace=True)
type_chart['Damage Multiplier'] = type_chart['Damage Multiplier'].replace('½', '0.5').astype(float)
type_chart.head(25)
type_chart[(type_chart["Atk. Move Type"] == "Electric") & (type_chart["Def. Pokemon Type"] == "Flying")]

In [ ]:
import pandas as pd
import numpy as np

moves_ds =pd.read_csv(r"DataBases\df_moves.csv")
moves_ds=moves_ds.fillna(0.0,inplace=False)


In [ ]:
import pandas as pd
import numpy as np

moves_ds =pd.read_csv(r"DataBases\df_moves.csv")
moves_ds.rename(columns={"Prob. (%)":"Prob"},inplace=True)
moves_with_effect=moves_ds.where(moves_ds["Prob"].notna()).dropna()
status_moves=moves_ds.where(moves_ds["Damage_class"]=="Status").dropna(how="all")
df=moves_with_effect.where(moves_with_effect["Prob"]==100).dropna(how="all")
#df["Effect"].str.contains("Special Defense").any()
moves_ds.where(moves_ds["Name"]=="Flare Blitz").dropna(how="all")

In [ ]:
import pandas as pd
pokemove = pd.read_csv(r"DataBases\\bridge_pokemon_moves_MAY_LEARN.csv")
pokemoves_dict = pokemove.groupby('Pokemon')['Move'].apply(list).to_dict()

def pokemon_with_move(desired_move):
    pokemon_with_desired_move = [pokemon for pokemon, moves in pokemoves_dict.items() if desired_move in moves]
    print(f"Pokemon that can learn '{desired_move}': {pokemon_with_desired_move}, {len(pokemon_with_desired_move)} pokemons")

pokemon_with_move("Beak Blast")

In [ ]:
import pandas as pd
pokemon_dt = pd.read_csv(r"DataBases\\df_pokemon.csv")
pokemon_dt=pokemon_dt.drop(columns=["Species",'Variant','Generation','Evolves_from','Has_gender_diff','VGC2022_rules','Monthly Usage (k)','Usage Percent (%)','Monthly Rank'])
pokemon_dt=pokemon_dt.set_index("ID")
pokemon_dt = pokemon_dt[~pokemon_dt["Name"].str.contains("Mega|Gigantamax")]
pokemon_dt = pokemon_dt[~pokemon_dt["Rarity"].str.contains("legendary|mythical|baby", case=False)]
pokemon_dt.head(50)

In [ ]:
import pandas as pd
import re

# Read your dataset
pokemove = pd.read_csv(r"DataBases\\bridge_pokemon_moves_MAY_LEARN.csv")
moves_ds = pd.read_csv(r"DataBases\df_moves.csv")
moves_ds.rename(columns={"Prob. (%)": "Prob"}, inplace=True)
moves_ds = moves_ds.dropna(subset=["Damage_class"])
moves_ds["Effect"] = moves_ds["Effect"].astype(str).str.lower()
moves_ds= moves_ds[~((moves_ds["Damage_class"].isin(["Special", "Physical"])) & (moves_ds["Power"].isna()))]

# Define regular expression patterns for different effect types
lower_stat_pattern = r"(?i)lowers? ?\w* (opponent's|user's|target's) (\w+)"
raise_stat_pattern = r"(?i)raises? ?\w* (opponent's|user's) (\w+)"
special_stat_pattern = r"(?i)lowers? ?\w* (opponent's|user's|target's) special (\w+)|raises? ?\w* (opponent's|user's|target's) special (\w+)"
hit_multiple= r"(?i)Hits (\d)-(\d) times"
status_condition= r"(?i)(poison|paralyze|burn|freeze|confuse)"
traps = r"(?i)traps? opponent"
recoil = r"(?i)receives? recoil damage"
atk_first=r"(?i)(attacks|goes) first"
high_crit=r"(?i)high critical hit ratio"
flinch=r"(?i)cause flinch"
drain=r"(?i)hp inflicted"
recover=r'(?i)recovers half its max hp'

# Define a function to translate move effects using regular expressions
def translate_effect(effect):
    lower_match = re.search(lower_stat_pattern, effect)
    raise_match = re.search(raise_stat_pattern, effect)
    special_match = re.search(special_stat_pattern, effect)
    hit_multiple_match = re.search(hit_multiple, effect)
    status_condition_match = re.search(status_condition, effect)
    traps_match = re.search(traps, effect)
    recoil_match = re.search(recoil, effect)
    atk_first_match = re.search(atk_first, effect)
    high_crit_match=re.search(high_crit,effect)
    flinch_match=re.search(flinch,effect)
    drain_match=re.search(drain,effect)
    recover_match=re.search(recover,effect)
    move_effect=""
    effects = []


    if special_match:
        whom = special_match.group(1) or special_match.group(3)
        #print(type(special_match.group(4)))
        stat = f"special {special_match.group(4)}" if special_match.group(2)==None else f"special {special_match.group(2)}"
        effect_type = "lower" if special_match.group(1) else "raise"
        whom = whom.replace("target", "opponent")
        effects.append(f"stat change: ({whom[:-2]},{effect_type},{stat})")
    elif raise_match:
        whom = raise_match.group(1)
        stat = raise_match.group(2)
        effect_type = "raise"
        whom = whom.replace("target", "opponent")
        effects.append(f"stat change: ({whom[:-2]},{effect_type},{stat})")
    elif lower_match:
        whom = lower_match.group(1)
        stat = lower_match.group(2)
        effect_type = "lower"
        whom = whom.replace("target", "opponent")
        effects.append(f"stat change: ({whom[:-2]},{effect_type},{stat})")

    if hit_multiple_match:
        effects.append("multihit")
    if status_condition_match:
        effects.append(status_condition_match.group(1))
    if traps_match:
        effects.append("traps")
    if recoil_match:
        effects.append("recoil")
    if atk_first_match:
        effects.append("first")
    if high_crit_match:
        effects.append("critical")
    if flinch_match:
        effects.append("flinch")
    if drain_match:
        effects.append("drain")
    if recover_match:
        effects.append("recover")

    # Combine all effects into a comma-separated string
    move_effect += ",".join(effects)
    if move_effect=="":
        move_effect="nan"

    return move_effect

# Apply the translation function to create the Move_effect column
moves_ds["Move_effect"] = moves_ds["Effect"].apply(translate_effect)
moves_ds = moves_ds.where((moves_ds["Move_effect"] != "nan") | (moves_ds["Damage_class"] != "Status")).dropna(how="all")

chosed_moves= pokemove.where(pokemove['Pokemon']=="Mew").dropna(how='all')
moveset = chosed_moves.merge(moves_ds,left_on="Move",right_on="Name")
moveset=moveset.drop(columns=['Pokemon','Move','Introducted_in'])
moves_ds.where(moves_ds["Move_effect"]=="burn").dropna(how="all")


In [15]:
from PIL import Image

# Open the image
image_path = "Assets\Type.png"
image = Image.open(image_path)

# Define the coordinates for each type
type_coordinates = {
    "Grass": (15, 205, 118, 245),
    "Fire": (15, 140, 118, 176),
    "Water": (271, 335, 373, 378),
    "Electric": (15, 74, 118, 110),
    "Ice": (271, 205, 373, 245),
    "Fighting": (271, 74, 373, 110), 
    "Poison": (144, 271, 248, 310),
    "Ground": (144, 205, 248, 245),
    "Flying": (144, 140, 248, 176),
    "Psychic": (271, 271, 373, 310),
    "Bug": (15, 10, 118, 47),
    "Rock": (15, 335, 118, 378),
    "Ghost": (271, 140, 373, 176),
    "Dragon": (271, 10, 373, 47),
    "Dark": (144, 10, 248, 47),
    "Steel": (144, 335, 248, 378),
    "Fairy": (144, 74, 248, 110),  
    "Normal": (15, 271, 118, 310)
}

# Crop and save each type image
output_folder = "Assets\Types"
for type_name, coordinates in type_coordinates.items():
    # Crop the image based on the coordinates
    cropped_image = image.crop(coordinates)

    # Save the cropped image
    output_path = f"{output_folder}/{type_name}.png"
    cropped_image.save(output_path)

    print(f"Saved {type_name} image at {output_path}")

Saved Grass image at Assets\Types/Grass.png
Saved Fire image at Assets\Types/Fire.png
Saved Water image at Assets\Types/Water.png
Saved Electric image at Assets\Types/Electric.png
Saved Ice image at Assets\Types/Ice.png
Saved Fighting image at Assets\Types/Fighting.png
Saved Poison image at Assets\Types/Poison.png
Saved Ground image at Assets\Types/Ground.png
Saved Flying image at Assets\Types/Flying.png
Saved Psychic image at Assets\Types/Psychic.png
Saved Bug image at Assets\Types/Bug.png
Saved Rock image at Assets\Types/Rock.png
Saved Ghost image at Assets\Types/Ghost.png
Saved Dragon image at Assets\Types/Dragon.png
Saved Dark image at Assets\Types/Dark.png
Saved Steel image at Assets\Types/Steel.png
Saved Fairy image at Assets\Types/Fairy.png
Saved Normal image at Assets\Types/Normal.png
